In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random
import jieba
from collections import Counter
%matplotlib inline

# Pattern Based AI

In [3]:
hello_rules = '''
say_hello = names hello tail
names = name names | name 
name = stink | mai | 老李
hello = hello | 早上好 | 你好 
tail = 呀 | !
'''

In [4]:
def make_sentence_by_gram(grammar_str:str,target,stmt_split='=',expr_split='|'):
    rules = dict()
    for line in grammar_str.split('\n'):
        if not line:continue
        stmt,expr = line.split(stmt_split)
        rules[stmt.strip()] = expr.split(expr_split)
    
    generated = generate(rules,target = target)
    
    return generated

In [47]:
def generate(grammer_rules,target):
    if target in grammer_rules:
        candidates = grammer_rules[target]
        candidate = random.choice(candidates)
        return ''.join(generate(grammer_rules,target = c.strip()) for c in candidate.split())
    else:
        return target

In [6]:
for i in range(3):
    print(make_sentence_by_gram(hello_rules,'say_hello'))

老李mai早上好!
stink老李mai老李maimai早上好呀
老李老李你好!


In [7]:
simple_grammar = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => Adj | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个
noun =>   女人 |  篮球 | 桌子 | 小猫
verb => 看着   |  坐在 |  听着 | 看见
Adj =>   蓝色的 |  好看的 | 小小的
"""

In [8]:
for i in range(3):
    print(make_sentence_by_gram(simple_grammar,target = 'sentence',stmt_split = '=>'))

这个小小的桌子听着一个蓝色的桌子
这个小小的蓝色的好看的桌子坐在这个小小的小猫
这个蓝色的小小的桌子看着这个好看的小小的蓝色的小猫


# Language Model  
## 2-Gram

$$ Pr(sentence)  = Pr( w_1 \cdot w_2 \cdots w_n) = \prod \frac{count(w_i,w_{i+1})} {count(w_i)} $$

In [9]:
text_path = '/Users/liyehong/Desktop/nlp/lession1/article_9k.txt'

In [10]:
FILE = open(text_path).read()

In [11]:
len(FILE)

33425826

In [12]:
FILE[:1000]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体验版内测稳定版暂不受影响以确保工程师可以集中全部精力进行系统优化工作有人猜测这也是将精力主要用到MIUI9的研发之中MIUI8去年5月发布距今已有一年有余也是时候更新换代了当然关于MIUI9的确切信息我们还是等待官方消息\n骁龙835作为唯一通过Windows10桌面平台认证的ARM处理器高通强调不会因为只考虑性能而去屏蔽掉小核心相反他们正联手微软找到一种适合桌面平台的兼顾性能和功耗的完美方案报道称微软已经拿到了一些新的源码以便Windows10更好地理解biglittle架构资料显示骁龙835作为一款集成了CPUGPU基带蓝牙WiFi的SoC比传统的Wintel方案可以节省至少30的PCB空间按计划今年Q4华硕惠普联想将首发骁龙835Win10电脑预计均是二合一形态的产品当然高通骁龙只是个开始未来也许还能见到三星Exynos联发科华为麒麟小米澎湃等进入Windows10桌面平台\n此前的一加3T搭载的是3400mAh电池DashCharge快充规格为5V4A至于电池缩水可能与刘作虎所说一加手机5要做市面最轻薄大屏旗舰的设定有关按照目前掌握的资料一加手机5拥有55寸1080P三星AMOLED显示屏6G8GBRAM64GB128GBROM双1600万摄像头备货量惊喜根据京东泄露的信息一加5起售价是xx99元应该是在279928992999中的某个\n这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车新华社记者张立云摄\n原标题44岁女子跑深圳约会网友被拒暴雨中裸身奔走深圳交警微博称昨日清晨交警发现有一女子赤裸上身行走在南坪快速上期间还起了轻生年头一辅警发现后赶紧为其披上黄衣并一路劝说她那么事发时到底都发生了些什么呢南都记者带您一起还原现场南都记者在龙岗大队坂田中队见到了辅警刘青发现女生的辅警一位外表高大帅气说话略带些腼腆的90后青年刘青介绍6月16日早上7时36分他正在环城南路附近值勤接到中队关于一位女子裸身进入机动车可能有危险的警情随后骑着小铁骑开始沿路寻找大概花了十多分钟在南坪大道坂田出口往龙岗方向的逆行辅道上发现该女子女子身上一丝不挂地逆车流而行时走时停时坐时躺险象环生刘青停好小铁骑和另外一名巡防员追了上去发现女子的情绪很低落话不多刘青尝试和女子交流劝说女子离开可女

In [13]:
max_length = 100000
sub_file = FILE[:max_length]

In [14]:
def cut(file):
    return list(jieba.cut(file))

In [15]:
TOKENS = cut(FILE)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3x/p6tf9w8s1z325p6pg0rzf59w0000gn/T/jieba.cache
Loading model cost 0.904 seconds.
Prefix dict has been built succesfully.


In [170]:
print(TOKENS)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
len(TOKENS)

17618254

In [17]:
words_count = Counter(TOKENS)

In [18]:
words_count.most_common(20)

[('的', 703716),
 ('n', 382020),
 ('在', 263597),
 ('月', 189330),
 ('日', 166300),
 ('新华社', 142462),
 ('和', 134061),
 ('年', 123106),
 ('了', 121938),
 ('是', 100909),
 ('\n', 89611),
 ('１', 88187),
 ('０', 84945),
 ('外代', 83268),
 ('中', 73926),
 ('中国', 71179),
 ('２', 70521),
 ('2017', 69894),
 ('记者', 62147),
 ('二线', 61998)]

In [174]:
# words_with_fre = [f for w , f in words_count.most_common()]

In [175]:
# print(words_with_fre[:10])

In [176]:
# plt.plot(np.log(np.log(words_with_fre)))

In [177]:
# list(jieba.cut('人的梦想是永远不会完结地'))

In [19]:
two_gram_words = [
    TOKENS[i] + TOKENS[i + 1] for i in range(len(TOKENS) - 1)
]

In [20]:
two_gram_words[:10]

['此外自', '自本周', '本周6', '6月', '月12', '12日起', '日起除', '除小米', '小米手机', '手机6']

In [21]:
two_gram_words_count = Counter(two_gram_words)

In [23]:
two_gram_words_count.most_common(20)

[('\n新华社', 69033),
 ('2017年', 61480),
 ('外代二线', 61301),
 ('n新华社', 59794),
 ('日n', 52216),
 ('新华社照片', 50401),
 ('5月', 37977),
 ('4月', 34571),
 ('新华社记者', 30864),
 ('２０', 27166),
 ('日在', 27154),
 ('年5', 25433),
 ('n当日', 25241),
 ('年4', 23727),
 ('n外代', 20854),
 ('照片外代', 20777),
 ('比赛中', 20637),
 ('外代2017', 20463),
 ('n5月', 20426),
 ('n4月', 19920)]

In [24]:
def get_one_gram_count(word):
    if word in words_count:
        return words_count[word]
    else:
        return words_count.most_common()[-1][-1]

In [25]:
print(get_one_gram_count('自然语言'))

42


In [26]:
def get_two_gram_count(word):
    if word in two_gram_words_count:
        return two_gram_words_count[word]
    else:
        return two_gram_words_count.most_common()[-1][-1]

In [27]:
print(get_two_gram_count('怎么说'))

41


In [28]:
def get_gram_count(word,wc):
    if word in wc:
        return wc[word]
    else:
        return wc.most_common()[-1][-1]

In [29]:
get_gram_count('你是谁',two_gram_words_count)

1

In [30]:
def two_gram_model(sentence):
    tokens = cut(sentence)
    
    probility = 1
    
    for i in range(len(tokens) - 1):
        word = tokens[i]
        next_word = tokens[i+1]
        
        two_gram_c = get_gram_count(word + next_word,two_gram_words_count)
        one_gran_c = get_gram_count(word,words_count)
        pro = two_gram_c / one_gran_c
        
        probility *= pro
    
    return probility

In [135]:
two_gram_model('白石麻衣天下第一')

0.14285714285714285

In [136]:
two_gram_model('第一白石麻衣天下')

3.4111065629690267e-05

In [33]:
two_gram_model('人的梦想是永远不会完结地')

2.918959982837208e-16

In [34]:
two_gram_model('人的是永远不会梦想完结地')

4.25106196201993e-17

# project-1 设计自己的句子生成器

## 场景：与朋友打电话

In [38]:
human = """
human = subject verb noun tail
subject = 你 | 兄弟 | 铁汁 | 老铁 | bro
verb = 在 | 想 | 有
noun = 吃饭 | 打游戏 | 看电影 | 学习 
tail = 吗 | ？
"""

In [39]:
make_sentence_by_gram(human,target = 'human')

'兄弟有看电影吗'

In [119]:
def generate_n(grammar_str:str,target,n=1,stmt_split='=',expr_split='|'):
    for i in range(n):
        sentence = make_sentence_by_gram(grammar_str,target,stmt_split='=',expr_split='|')
        print(sentence)

In [50]:
generate_n(human,target='human',n=10)

兄弟有吃饭？
兄弟有打游戏吗
兄弟有学习？
你想打游戏？
bro想学习吗
你有吃饭吗
你在吃饭吗
老铁在看电影？
你在看电影吗
铁汁想看电影吗


In [61]:
friend = """
friend = hello separate name separate answer separate sorry separate end tail
hello = hi | hello | hey
name = stink | my bro | 铁汁 | 老李
separate = , | ! | .
answer = 我要 reason 
reason = 帮妈妈做家务 | 照顾弟弟 | 去外婆家 | 外出旅行 | 
sorry = 抱歉 | 对不起 | 不好意思
end = 下次约 | 忙完联系你
tail = !!!
"""

In [62]:
generate_n(friend,target='friend',n=5)

hi!铁汁.我要去外婆家!对不起!忙完联系你!!!
hey.铁汁!我要帮妈妈做家务.不好意思,忙完联系你!!!
hi!老李.我要外出旅行.对不起,下次约!!!
hey,mybro,我要照顾弟弟,抱歉.忙完联系你!!!
hi!stink,我要去外婆家!不好意思,下次约!!!


# project-2 使用新数据源完成语言模型的训练

In [63]:
train_txt_path = '/Users/liyehong/Desktop/nlp/lession1/train.txt'

In [64]:
train_file = open(train_txt_path).read()

In [65]:
len(train_file)

1343520

In [66]:
train_file[:1000]

'0 ++$++ disability-insurance ++$++ 法律要求残疾保险吗？ ++$++ Is  Disability  Insurance  Required  By  Law?\n1 ++$++ life-insurance ++$++ 债权人可以在死后人寿保险吗？ ++$++ Can  Creditors  Take  Life  Insurance  After  Death?\n2 ++$++ renters-insurance ++$++ 旅行者保险有租赁保险吗？ ++$++ Does  Travelers  Insurance  Have  Renters  Insurance?\n3 ++$++ auto-insurance ++$++ 我可以开一辆没有保险的新车吗？ ++$++ Can  I  Drive  A  New  Car  Home  Without  Insurance?\n4 ++$++ life-insurance ++$++ 人寿保险的现金转出价值是否应纳税？ ++$++ Is  The  Cash  Surrender  Value  Of  Life  Insurance  Taxable?\n5 ++$++ annuities ++$++ 如何报告年金收入？ ++$++ How  Is  Annuity  Income  Reported?\n6 ++$++ home-insurance ++$++ AAA家庭保险涵盖什么？ ++$++ What  Does  AAA  Home  Insurance  Cover?\n7 ++$++ retirement-plans ++$++ 什么是简单的退休计划？ ++$++ What  Is  A  Simple  Retirement  Plan?\n8 ++$++ disability-insurance ++$++ 社会保险残疾保险是什么？ ++$++ What  Does  Social  Security  Disability  Insurance  Cover?\n9 ++$++ auto-insurance ++$++ 汽车保险是否预付？ ++$++ Is  Car  Insurance  Prepaid?\n10 ++$++ medicare-ins

In [67]:
import re

In [68]:
real_text = re.compile(r'[\u4e00-\u9fa5]+')

In [69]:
train_result = real_text.findall(train_file)

In [76]:
ss = ''.join(line for line in train_result)

In [83]:
train_tokens = cut(ss) # one-gram 分词

In [84]:
len(train_tokens)

74259

In [85]:
train_words_count = Counter(train_tokens) # 统计one-gram分词中各个词在文本中出现的频次

In [87]:
train_words_count.most_common(20)

[('保险', 5005),
 ('的', 3220),
 ('人寿保险', 2962),
 ('什么', 2675),
 ('吗', 2479),
 ('是', 2344),
 ('我', 2053),
 ('是否', 1862),
 ('可以', 1704),
 ('健康', 1513),
 ('如何', 1294),
 ('医疗保险', 1269),
 ('多少', 1244),
 ('汽车保险', 1189),
 ('在', 913),
 ('覆盖', 847),
 ('你', 827),
 ('有', 770),
 ('残疾', 724),
 ('房主', 714)]

In [93]:
train_tokens_two = [
    train_tokens[i] + train_tokens[i+1] for i in range(len(train_tokens) - 1)
]                     # two-gram 分词               

In [94]:
train_words_count_two = Counter(train_tokens_two)

In [95]:
train_words_count_two.most_common(20)  # 统计two-gram分词中各个词在文本中出现的频次

[('健康保险', 1347),
 ('什么是', 1152),
 ('保险是否', 975),
 ('我的', 726),
 ('残疾保险', 658),
 ('房主保险', 602),
 ('我可以', 530),
 ('保险吗', 511),
 ('是否覆盖', 504),
 ('家庭保险', 440),
 ('年金', 427),
 ('长期护理', 416),
 ('是什么', 380),
 ('护理保险', 367),
 ('的人寿保险', 319),
 ('吗什么', 317),
 ('人寿保险吗', 312),
 ('你可以', 299),
 ('退休计划', 289),
 ('保险的', 280)]

In [ ]:
def get_gram_count(word,wc):
    if word in wc:
        return wc[word]
    else:
        return wc.most_common()[-1][-1]

In [96]:
def insurance_two_gram_model(sentence):
    sentence_tokens = cut(sentence)
    
    probability = 1
    
    for i in range(len(sentence_tokens) - 1):
        word = sentence_tokens[i]
        next_word = sentence_tokens[i+1]
        
        one_gram_count = get_gram_count(word,train_words_count)
        two_gram_count = get_gram_count(next_word,train_words_count_two)
        pro = two_gram_count / one_gram_count
        
        probability *= pro
        
    return probability

In [97]:
insurance_two_gram_model('汽车保险是否预付')

4.516879125604471e-07

In [98]:
insurance_two_gram_model('汽车预付保险是否')

2.4666691333358e-07

In [99]:
insurance_two_gram_model('法律要求残疾保险吗')

1.6223816978004468e-10

In [106]:
insurance_two_gram_model('法律是否大家可好看要求残疾保险')

1.06363736835441e-11

In [107]:
insurance_two_gram_model('我今天早上没有吃早饭')

2.2655444669740255e-06

In [109]:
insurance_two_gram_model('我今天早上没有吃晚饭和早饭')

1.5003605741549836e-08

## project-3 获取最优质的的语言

In [115]:
sentence_rules = """
sentence_rules = name date staus place event
name = stink | mai | 寅子 | lex | 老李
date = year month day 
year = 2019年 | 2018年 | 2017年
month = 1月 | 2月 | 3月 | 4月 | 5月 | 6月 | 7月 | 8月 | 9月 | 10月 | 11月 | 12月 | 13月 | 14月
day = 1日 | 2日 | 3日 | 4日 | 5日 | 6日 | 7日 | 31日 | 32日 | 33日 |
staus = 在 | 去 | 回 | 到 | 爬 | 跑 | 走
place = 上海 | 北京 | 深圳 | 长沙
event = 出差 | 吃早饭 | 买房 | 下棋 | 打游戏 | 旅行 | 游泳 | 上学 | 谈恋爱 | 唱歌 | 回家 | 战狼 | 洗澡 | 自然语言处理
"""

In [116]:
generate_n(sentence_rules,target='sentence_rules',n=10)

stink2017年6月到深圳吃早饭
lex2017年9月31日爬北京买房
lex2018年2月7日回北京买房
寅子2019年4月5日在长沙买房
lex2017年13月7日在深圳战狼
stink2017年1月回北京打游戏
老李2019年14月2日走北京出差
stink2019年13月5日走长沙洗澡
老李2017年4月7日在上海买房
寅子2018年7月33日到深圳回家


In [122]:
def generate_n_list(grammar_str:str,target,n=1,stmt_split='=',expr_split='|'):
    sentence_list = []
    for i in range(n):
        sentence = make_sentence_by_gram(grammar_str,target,stmt_split='=',expr_split='|')
        sentence_list.append(sentence)
    return sentence_list

In [123]:
sentence_l = generate_n_list(sentence_rules,target='sentence_rules',n=10)

In [124]:
print(sentence_l)

['老李2019年12月6日爬深圳打游戏', 'mai2018年10月32日在长沙自然语言处理', '老李2017年14月7日在上海回家', 'stink2017年4月2日走深圳游泳', '寅子2017年12月4日在上海吃早饭', '老李2017年4月5日去上海上学', '老李2019年8月6日在深圳打游戏', '寅子2019年11月4日到上海战狼', 'stink2017年9月4日爬北京下棋', 'mai2018年8月2日回深圳谈恋爱']


In [ ]:
def two_gram_model(sentence):
    tokens = cut(sentence)
    
    probility = 1
    
    for i in range(len(tokens) - 1):
        word = tokens[i]
        next_word = tokens[i+1]
        
        two_gram_c = get_gram_count(word + next_word,two_gram_words_count)
        one_gran_c = get_gram_count(word,words_count)
        pro = two_gram_c / one_gran_c
        
        probility *= pro
    
    return probility

In [157]:
def generate_best(sentence_rules,target,n=1,stmt_split='=',expr_split='|'):
    sentence_l = generate_n_list(sentence_rules,target,n,stmt_split,expr_split)
    
    sentence_probility_list = []
    
    for s in sentence_l:
        probility = two_gram_model(s)
        t = (s , probility)
#         print(t)
        sentence_probility_list.append(t)
#     print(sentence_probility_list)
    new_list = sorted(sentence_probility_list , key = lambda x : x[1] , reverse=True)
#     print(new_list)
    return new_list[0]

In [158]:
generate_best(sentence_rules,target='sentence_rules',n=10)

('stink2019年6月5日爬上海出差', 1.928318247799569e-16)
('stink2018年6月2日在上海唱歌', 1.3353989616287027e-11)
('老李2018年14月2日走北京战狼', 1.5591373088454192e-21)
('lex2018年14月31日爬北京回家', 1.6533598219244346e-20)
('stink2019年12月1日去深圳洗澡', 4.917837787406215e-13)
('mai2017年10月31日跑北京打游戏', 3.3245354428453914e-18)
('老李2019年6月7日回长沙打游戏', 4.8311730606388784e-11)
('寅子2017年13月2日在长沙唱歌', 1.449909156538374e-16)
('mai2017年11月6日走深圳下棋', 1.7690381773611766e-17)
('mai2019年6月31日在北京吃早饭', 1.9293890939568776e-13)
[('老李2019年6月7日回长沙打游戏', 4.8311730606388784e-11), ('stink2018年6月2日在上海唱歌', 1.3353989616287027e-11), ('stink2019年12月1日去深圳洗澡', 4.917837787406215e-13), ('mai2019年6月31日在北京吃早饭', 1.9293890939568776e-13), ('stink2019年6月5日爬上海出差', 1.928318247799569e-16), ('寅子2017年13月2日在长沙唱歌', 1.449909156538374e-16), ('mai2017年11月6日走深圳下棋', 1.7690381773611766e-17), ('mai2017年10月31日跑北京打游戏', 3.3245354428453914e-18), ('lex2018年14月31日爬北京回家', 1.6533598219244346e-20), ('老李2018年14月2日走北京战狼', 1.5591373088454192e-21)]


('老李2019年6月7日回长沙打游戏', 4.8311730606388784e-11)